# Create a CAIC weather data map 

## 1. Install requirements 

In [242]:
%pip install folium datetime lxml


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. Import requirements

In [243]:
import folium
import datetime
import os
import json
from folium.plugins import GroupedLayerControl
from folium.plugins import MeasureControl
from folium.plugins import MousePosition
from folium.plugins import MarkerCluster
from folium.plugins import FeatureGroupSubGroup
from folium.elements import Element
from folium import MacroElement
from jinja2 import Template
from bs4 import BeautifulSoup


## 3. Generate Functions
- The first function collections the current date and time for use later on url creation. 
- The second function generates urls that are the base url for different CAIC products.

In [244]:
def get_current_datetime():
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d+%H:%M")

def generate_station_url(station_code, station_title, elevation, url_type):
    current_datetime = get_current_datetime()
    title_encoded = station_title.replace(" ", "+")
    
    if url_type == 'weather':
        return f"https://classic.avalanche.state.co.us/caic/obs_stns/station.php?plot=hourly&st={station_code}&date={current_datetime}&unit=e&area=caic&title={title_encoded}"
    elif url_type == 'windrose':
        return f"https://classic.avalanche.state.co.us/caic/obs_stns/windrose.php?st={station_code}&date={current_datetime}&elev={elevation}&unit=e&area=caic&title={title_encoded}"
    elif url_type == 'plot':
        return f"https://classic.avalanche.state.co.us/caic/obs_stns/hplot.php?title={title_encoded}&st={station_code}&date={current_datetime}&unit=e&area=caic&range=48"
    else:
        raise ValueError(f"Invalid URL type: {url_type}")

In [245]:
def load_json_data(json_file_path):
    """
    Generic function to load JSON data from a file.
    Returns the data if successful, else returns None.
    """
    if not os.path.exists(json_file_path):
        print(f"Error: The file '{json_file_path}' does not exist.")
        return None
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"Successfully loaded data from '{json_file_path}'.")
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from '{json_file_path}': {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while loading '{json_file_path}': {e}")
        return None


In [246]:
class CustomMapClickPopup(MacroElement):
    """
    A custom Folium plugin to handle map clicks and display a popup
    with latitude, longitude, and a dynamic "View Forecast" link.
    """
    _template = Template("""
        {% macro script(this, kwargs) %}
            function onMapClick(e) {
                var lat = e.latlng.lat.toFixed(6);
                var lon = e.latlng.lng.toFixed(6);
                var idate = "{{ this.idate }}";
                var res = {{ this.res }};
                var url = "{{ this.forecast_url }}" + "?idate=" + idate + "&res=" + res + "&lat=" + lat + "&lon=" + lon;
                
                var popupContent = `
                    <div>
                        <b>Coordinates:</b> ${lat}, ${lon}<br>
                        <a href="${url}" target="_blank">View CAIC WRF Forecast</a>
                    </div>
                `;
                
                L.popup()
                    .setLatLng(e.latlng)
                    .setContent(popupContent)
                    .openOn({{ this._parent.get_name() | safe }});
            }

            {{ this._parent.get_name() | safe }}.on('click', onMapClick);
        {% endmacro %}
    """)

    def __init__(self, idate, forecast_url, res=4):
        """
        Initialize the custom popup handler.

        :param idate: The current date as a string (e.g., "2024-04-27+15:30")
        :param forecast_url: The base URL for the forecast (e.g., "https://example.com/forecast")
        :param res: Resolution parameter (default is 4)
        """
        super(CustomMapClickPopup, self).__init__()
        self.idate = idate
        self.forecast_url = forecast_url
        self.res = res

    def render(self, **kwargs):
        super(CustomMapClickPopup, self).render()


## 4. Define Folium function to generate the map

In [247]:
def weather_map(stations, forecast_locations, webcam_locations, forecast_url, center_lat=39.5501, center_lon=-106.0667, zoom_start=7):

    current_date = get_current_datetime()
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, control_scale=True)

    # Define a list of tile layers with their configurations
    #tile_layers = [
        #{
        #    'tiles': 'openstreetmap',
        #    'name': 'OpenStreetMap',
        #    'attr': None  
        #},
        # Add more tile layers if needed. Can add imagery easily below
        # {
        #    'tiles': 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        #    'name': 'Esri World Imagery',
        #    'attr': 'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
        #}
    #]

    # Loop through the tile list and add each tile layer to the map
    #for layer in tile_layers:
        #if layer['tiles'] == 'openstreetmap':
        #    folium.TileLayer(layer['tiles'], name=layer['name']).add_to(m)
        #else:
        #    folium.TileLayer(
        #        tiles=layer['tiles'],
        #        name=layer['name'],
        #        attr=layer['attr']
        #    ).add_to(m)
    
    # Define main FeatureGroups for grouping 
    zones = sorted(list({station['zone'] for station in stations if 'zone' in station}))
    zone_feature_groups = {}
    for zone in zones:
        fg = folium.FeatureGroup(name=zone, show=False)
        zone_feature_groups[zone] = fg
        fg.add_to(m)
    
    data_group = folium.FeatureGroup(name='Data').add_to(m)
    overlays_group = folium.FeatureGroup(name='Overlays').add_to(m)

    # Define SubGroups for Data
    webcams_fg = FeatureGroupSubGroup(data_group, 'Webcams', show=False)
    forecasts_fg = FeatureGroupSubGroup(data_group, 'Forecasts', show=False)

    # Define SubGroups for Overlays
    noaaradar_fg = FeatureGroupSubGroup(overlays_group, 'NOAA Radar', show=False)
    kftg_radar_fg = FeatureGroupSubGroup(overlays_group, 'KFTG', show=False)
    kgjx_radar_fg = FeatureGroupSubGroup(overlays_group, 'KGJX', show=False)
    kpux_radar_fg = FeatureGroupSubGroup(overlays_group, 'KPUX', show=False)
    satellite_fg = FeatureGroupSubGroup(overlays_group, 'GOES Vis', show=False)
    watervapor_fg = FeatureGroupSubGroup(overlays_group, 'GOES Vapor', show=False)
    sfc_map_fg = FeatureGroupSubGroup(overlays_group, 'Sfc Map', show=False)

    # Add feature groups to the map
    webcams_fg.add_to(m)
    forecasts_fg.add_to(m)
    noaaradar_fg.add_to(m)
    kftg_radar_fg.add_to(m)
    kgjx_radar_fg.add_to(m)
    kpux_radar_fg.add_to(m)
    satellite_fg.add_to(m)
    watervapor_fg.add_to(m)
    sfc_map_fg.add_to(m)
    
    folium.WmsTileLayer(
        url="https://opengeo.ncep.noaa.gov/geoserver/conus/conus_bref_qcd/ows?",
        name="CONUS Radar Base Reflectivity",
        fmt="image/png",
        layers="conus_bref_qcd",
        styles="radar_reflectivity",
        attr="Weather data © NCEP",
        transparent=True,
        overlay=True,
        control=False,
    ).add_to(noaaradar_fg)
    
    
    
        # Add the KFTG SR Base Reflectivity layer to the 'KFTG Radar' FeatureGroup
    folium.WmsTileLayer(
        url="https://opengeo.ncep.noaa.gov/geoserver/kftg/ows?",
        name="KFTG SR Base Reflectivity",
        fmt="image/png",
        layers="kftg_sr_bref",
        styles="radar_reflectivity",
        attr="Weather data © NCEP",
        transparent=True,
        overlay=True,
        control=False,
    ).add_to(kftg_radar_fg)


    
        # Add the KGJX SR Base Reflectivity layer to the 'KGJX Radar' FeatureGroup
    folium.WmsTileLayer(
        url="https://opengeo.ncep.noaa.gov/geoserver/kgjx/ows?",
        name="KGJX SR Base Reflectivity",
        fmt="image/png",
        layers="kgjx_sr_bref",
        styles="radar_reflectivity",
        attr="Weather data © NCEP",
        transparent=True,
        overlay=True,
        control=False,
    ).add_to(kgjx_radar_fg)

    

    # Add the KPUX SR Base Reflectivity layer to the 'KPUX Radar' FeatureGroup
    folium.WmsTileLayer(
        url="https://opengeo.ncep.noaa.gov/geoserver/kpux/ows?",
        name="KPUX SR Base Reflectivity",
        fmt="image/png",
        layers="kpux_sr_bref",
        styles="radar_reflectivity",
        attr="Weather data © NCEP",
        transparent=True,
        overlay=True,
        control=False,
    ).add_to(kpux_radar_fg)
    
    # Add the GOES Visible Imagery layer to the 'Satellite Imagery' FeatureGroup
    folium.WmsTileLayer(
        url="https://nowcoast.noaa.gov/geoserver/observations/satellite/ows?",
        name="GOES East & West Satellite Visible Imagery",
        fmt="image/png",
        layers="goes_visible_imagery",
        styles="goes-vis",
        attr="Satellite data © NOAA/NESDIS",
        transparent=True,
        overlay=True,
        control=False,
    ).add_to(satellite_fg)

    # Add the GOES Water Vapor Imagery layer
    folium.WmsTileLayer(
        url="https://nowcoast.noaa.gov/geoserver/observations/satellite/ows?",
        name="GOES East & West Satellite Water Vapor Imagery",
        fmt="image/png",
        layers="goes_water_vapor_imagery",
        styles="goes-wv",
        attr="Satellite data © NOAA/NESDIS",
        transparent=True,
        overlay=True,
        control=False,
    ).add_to(watervapor_fg)

    # Add National Forecast Map 
    folium.WmsTileLayer(
        url="https://mapservices.weather.noaa.gov/vector/services/outlooks/natl_fcst_wx_chart/MapServer/WMSServer?",
        name="Day 1 Fronts", 
        layers="34",  # Layer Name from the XML
        fmt="image/png",
        styles="default",
        transparent=True,
        attr="Forecast data © NOAA/NWS",
        overlay=True,
        control=False,
    ).add_to(sfc_map_fg)

    
    # Define a color mapping for different zones
    zone_color_mapping = {
        'Steamboat': 'blue',
        'Front Range': 'red',
        'Vail/Summit': 'green',
        'Sawatch': 'purple',
        'Sangre': 'orange',
        'Grand Mesa': 'darkred',
        'North San Juan': 'cadetblue',
        'South San Juan': 'darkpurple',
        'Aspen': 'pink',
        'Gunnison': 'darkgreen'
        # Add more zones and colors as needed
    }
    
    # Add weather stations - Pulls from stations list to build map objects 
    # Stations will display links to current weather, current wind rose (link will display even if n/a), a current graph plot, and link to graph plot
    for station in stations:
        lat = station.get('latitude')
        lon = station.get('longitude')
        zone = station.get('zone', 'Unknown')
        
        if lat is None or lon is None:
            print(f"Warning: Station '{station.get('name', 'N/A')}' is missing 'latitude' or 'longitude'. Skipping.")
            continue  # Skip this station and move to the next
        
        weather_url = generate_station_url(station['code'], station['title'], station['elevation'], 'weather')
        windrose_url = generate_station_url(station['code'], station['title'], station['elevation'], 'windrose')
        plot_url = generate_station_url(station['code'], station['title'], station['elevation'], 'plot')
            
        color = zone_color_mapping.get(zone, 'blue')  # Default to 'blue' if zone not in mapping
            
        popup_content = f"""
            <div style="width:300px;">
                <b>{station['name']}</b><br>
                Provider: {station['provider']}<br>
                Lat: {station['latitude']}, Lon: {station['longitude']}<br>
                Elevation: {station['elevation']} ft<br>
                <b>Zone:</b> {zone}<br>
                <a href="{weather_url}" target="_blank">Current Weather</a><br>
                <a href="{windrose_url}" target="_blank">Current Windrose</a><br>
                <img src="{plot_url}" alt="Weather Plot" style="width:100%; max-width:100%;" loading="lazy"><br>
                <a href="{plot_url}" target="_blank">Open Full Weather Plot</a>
            </div>
            """
            
        marker =folium.Marker(
            location=[station['latitude'], station['longitude']],
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=station['name'],
            icon=folium.Icon(color=color, icon='info-sign') # Set color based on zone
        )
        
        # Add marker to the appropriate zone group
        if zone in zone_feature_groups:
            zone_feature_groups[zone].add_child(marker)
        else:
            # If zone not predefined, add to an 'Unknown' group
            if 'Unknown' not in zone_feature_groups:
                unknown_fg = folium.FeatureGroup(name='Unknown', show=False)
                zone_feature_groups['Unknown'] = unknown_fg
                unknown_fg.add_to(m)
            zone_feature_groups['Unknown'].add_child(marker)
        
        
    # Add forecast locations 
    # Forecast graphs have a current static link that is linked directly for three models 
    for location in forecast_locations:
        popup_content = f"""
        <div>
            <b>{location['name']}</b><br>
            Lat: {location['latitude']}, Lon: {location['longitude']}<br>
            Model Elevation: {location['model_elevation']} ft<br>
            Forecast Images:<br>
            <img src="{location['forecast_image_urls']['WRF']}" alt="WRF Forecast Graph" style="width:100%; max-width:100%;" loading="lazy"><br>
            <a href="{location['forecast_image_urls']['WRF']}" target="_blank">WRF</a> |
            <a href="{location['forecast_image_urls']['WRFHR']}" target="_blank">WRFHR</a> |
            <a href="{location['forecast_image_urls']['NAM']}" target="_blank">NAM</a>
        </div>
        """

        folium.Marker(
            location=[location['latitude'], location['longitude']],
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=location['name'],
            icon=folium.Icon(color='red', icon='cloud')
        ).add_to(forecasts_fg)

    # Add webcam locations 
    # Displays image and link to static webcams 
    for name, webcam in webcam_locations.items():
        # Initialize image HTML
        images_html = ""
        for img_url in webcam.get('image_urls', []):
            images_html += f"""
            <a href="{img_url}" target="_blank">
                <img src="{img_url}" alt="{name} Webcam"
                    style="width:100%; height:150px; object-fit:cover; cursor:pointer;"
                    title="Click to open full-size image" loading="lazy">
            </a>
            <br>
            """
    
        popup_content = f"""
        <div style="width:400px; height:auto;">
            <b>{name}</b><br>
            Lat: {webcam['latitude']}, Lon: {webcam['longitude']}<br>
            {images_html}
            <p><small>Click the images to open in full size</small></p>
        </div>
        """

        folium.Marker(
            location=[webcam['latitude'], webcam['longitude']],
            popup=folium.Popup(popup_content, max_width=400),
            tooltip=name,
            icon=folium.Icon(color='green', icon='camera')
        ).add_to(webcams_fg)
    



    # Add GroupedLayerControl

    groups_dict = {
        'Zones': [zone_feature_groups[zone] for zone in zone_feature_groups],
        'Data': [webcams_fg, forecasts_fg],
        'Overlays': [satellite_fg, watervapor_fg, sfc_map_fg, noaaradar_fg, kftg_radar_fg, kgjx_radar_fg, kpux_radar_fg]
    }

    GroupedLayerControl(
        groups=groups_dict,
        exclusive_groups=False,  # Allows multiple groups to be active simultaneously
        collapsed=False         # Keeps the layer control panel expanded
    ).add_to(m)
    
    title_html = '''<h3 align="center" style="font-size:16px"><b>CAIC Weather Stations, Forecast Locations and Webcams</b></h3>'''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Add the Custom Map Click Popup
    custom_click_popup = CustomMapClickPopup(idate=current_date, forecast_url=forecast_url, res=4)
    m.add_child(custom_click_popup)
    
    # Add Lat/long pop up marker 
    #m.add_child(folium.LatLngPopup())
    
    # Add measure tool
    #m.add_child(MeasureControl())

    # Add mouse position
    #MousePosition().add_to(m)

    return m

## 5. Object Data 
- Weather station data and url codes
- Forecast locations and direct urls
- Webcam locations and direct urls

In [248]:
if __name__ == "__main__":
    # Define the paths to your JSON files
    stations_file = os.path.join('data', 'stations.json')
    forecast_file = os.path.join('data', 'forecast_locations.json')
    webcam_file = os.path.join('data', 'webcam_locations.json')


    # Load data from JSON files
    stations = load_json_data(stations_file)
    forecast_locations = load_json_data(forecast_file)
    webcam_locations = load_json_data(webcam_file)

    # Check if all data is loaded successfully
    if stations is None or forecast_locations is None or webcam_locations is None:
        print("Error: One or more data files failed to load. Exiting.")
        exit(1)
        
    # Define point forecast base url 
    pt_forecast_url = "https://looper.avalanche.state.co.us/iptfcst/ptfcst.php" 



Successfully loaded data from 'data/stations.json'.
Successfully loaded data from 'data/forecast_locations.json'.
Successfully loaded data from 'data/webcam_locations.json'.


## 6. Run to create an html file

In [249]:
# Create the enhanced map
create_map = weather_map(stations, forecast_locations, webcam_locations, forecast_url=pt_forecast_url)

# Save the map to an HTML file
create_map.save("weather_map.html")

print("Weather Map has been created and saved as 'weather_map.html'")
    
    

Weather Map has been created and saved as 'weather_map.html'


## 7. Test map in notebook

In [250]:
create_map